In [1]:
!pip install experta
!pip install frozendict==2.3.7

  Preparing metadata (setup.py) ... done
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3149 sha256=aa6f766eef1d526a738f6089eb8aec5220ef300d47b313c70010bde0c7c22519
  Stored in directory: /root/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c3c2d3d2bf448fd6e
Successfully built frozendict
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.6
    Uninstalling frozendict-2.4.6:
      Successfully uninstalled frozendict-2.4.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.59 requires frozendict>=2.3.4, but you have frozendict 1.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for frozendict: filename=frozendict-2.3.7-py3-none-any.whl size=13594 sha256=6a1647420db59b

In [14]:
policies_df

,Category,Condition,max,Policy Description,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Credit Limit,CGPA < 2.00,12,"Fall, Spring",,,
1,Credit Limit,2.00 ≤ CGPA < 3.00,20,"Fall, Spring",,,
2,Credit Limit,CGPA ≥ 3.00,22,"Fall, Spring",,,
3,Credit Limit,Summer Semester,9,Summer,,,
4,Retaking Failed Courses,Grade = F,B+,Retaking a failed course is mandatory,,,


In [21]:
import pandas as pd

# Read the course file
courses_df = pd.read_csv("coruse.csv").fillna("")
policies_df = pd.read_csv("policies.csv").fillna("")


import pandas as pd
from experta import *
import re

courses_df = pd.read_csv("coruse.csv").fillna("")
policies_df = pd.read_csv("policies.csv").fillna("")

# Student input
student_input = {
     "cgpa": 3.2,                      # Credit limit = 22
    "semester": "FALL",
    "passed_courses": ["UC1"],       # Only 1 course passed
    "failed_courses": []
}

# Convert course list to dict
all_courses = courses_df.to_dict(orient="records")

# Define Fact model
class StudentProfile(Fact):
    pass

# Inference engine
class AdvisingEngine(KnowledgeEngine):
    def __init__(self, courses, student_data, policies_df):
        super().__init__()
        self.courses = courses
        self.student_data = student_data
        self.policies_df = policies_df
        self.recommended_courses = []
        self.total_credits = 0
        self.explanations = []
        self.credit_limit = self.get_dynamic_credit_limit()

    def get_dynamic_credit_limit(self):
        credit_policies = self.policies_df[self.policies_df["Category"].str.strip() == "Credit Limit"]
        rules = []

        for _, row in credit_policies.iterrows():
            condition = str(row["Condition"])
            max_credit = int(row["max"])
            match = re.findall(r'(\d+\.\d+)', condition)

            if "≥" in condition or ">=" in condition:
                rules.append((lambda cgpa, val=float(match[0]): cgpa >= val, max_credit))
            elif "≤" in condition and len(match) == 2:
                l, u = float(match[0]), float(match[1])
                rules.append((lambda cgpa, l=l, u=u: l <= cgpa < u, max_credit))
            elif "<" in condition:
                rules.append((lambda cgpa, val=float(match[0]): cgpa < val, max_credit))

        for rule, limit in rules:
            if rule(self.student_data["cgpa"]):
                return limit
        return 12  # Default fallback

    @Rule(StudentProfile())
    def recommend_courses(self):
        already_added = set()
        passed = [c.strip() for c in self.student_data["passed_courses"]]
        failed = [c.strip() for c in self.student_data["failed_courses"]]

        # Step 1: Prioritize failed courses
        for course in self.courses:
            code = str(course["Course Code"]).strip()

            if code not in failed:
                continue
            if code in passed:
                self.explanations.append(f"{code} is not recommended because it was already passed.")
                continue
            if self.student_data["semester"].upper() not in offered and offered != "BOTH":
                self.explanations.append(f"{code} is unavailable this semester.")
                continue
            if any(pr not in passed for pr in prereqs):
                self.explanations.append(f"{code} is not recommended due to unmet prerequisite(s): {', '.join([pr for pr in prereqs if pr not in passed])}.")
                continue
            if self.total_credits + credits > self.credit_limit:
                self.explanations.append(f"{code} is not added because it would exceed the credit limit.")
                continue
            self.recommended_courses.append(course)
            already_added.add(code)
            self.total_credits += credits
            self.explanations.append(f"{code} is prioritized because you failed it previously and met its prerequisites.")

        # Recommend other eligible courses
        for course in self.courses:
            code = str(course["Course Code"]).strip()
            name = str(course["Course Name"]).strip()
            offered = str(course["Semester Offered"]).strip().upper()
            prereqs = [p.strip() for p in str(course["Prerequisites"]).split(",") if p.strip()]
            coreqs = [c.strip() for c in str(course["Co-requisites"]).split(",") if c.strip()]
            credits = int(course["Credit Hours"])

            if code in passed or code in already_added:
                continue
            if self.student_data["semester"].upper() not in offered and offered != "BOTH":
                self.explanations.append(f"{code} is not offered in the {self.student_data['semester']} semester.")
                continue
            if any(pr not in passed for pr in prereqs):
                self.explanations.append(f"{code} is not recommended due to unmet prerequisite(s): {', '.join([pr for pr in prereqs if pr not in passed])}.")
                continue
            if any(cr not in passed and cr not in [c["Course Code"].strip() for c in self.recommended_courses] for cr in coreqs):
                self.explanations.append(f"{code} is not recommended due to unmet co-requisite(s): {', '.join([cr for cr in coreqs if cr not in passed])}.")
                continue
            if self.total_credits + credits > self.credit_limit:
                self.explanations.append(f"{code} is not added because it would exceed the credit limit.")
                continue
            self.recommended_courses.append(course)
            already_added.add(code)
            self.total_credits += credits
            if prereqs:
                self.explanations.append(f"{code} is recommended because you passed {', '.join(prereqs)}, its prerequisite(s).")
            else:
                self.explanations.append(f"{code} is recommended because it has no prerequisites.")

# Run the engine
engine = AdvisingEngine(all_courses, student_input, policies_df)
engine.reset()
engine.declare(StudentProfile(**student_input))
engine.run()

# Show output
recommended_df = pd.DataFrame(engine.recommended_courses)[["Course Code", "Course Name", "Credit Hours"]]
if not recommended_df.empty:
    recommended_df["Credit Hours"] = recommended_df["Credit Hours"].astype(int)
    recommended_df.loc["Total"] = ["", "Total Credits", recommended_df["Credit Hours"].sum()]
    print("Recommended Courses:")
    print(recommended_df)
else:
    print("No courses could be recommended based on your profile.")

# Show explanations
print("\n--- Explanation of Decisions ---")
for explanation in engine.explanations:
    print("- " + explanation)

Recommended Courses:
      Course Code                           Course Name  Credit Hours
0         MAT111                         Mathematics I              3
1          MAT123                             Mechanics             3
2         MEC011               Engineering Drawing (1)              3
3         PHY212   Introduction to Engineering Physics              3
4         CSE014               Structured Programming               3
5          MAT212                       Linear Algebra              3
6         CSE131                         Logic Design               3
Total                                     Total Credits            21

--- Explanation of Decisions ---
- MAT111 is recommended because it has no prerequisites.
- MAT123 is recommended because it has no prerequisites.
- MEC011 is recommended because it has no prerequisites.
- PHY212 is recommended because it has no prerequisites.
- CSE014 is recommended because it has no prerequisites.
- MAT112 is not offered in the